In [2]:
import os
import faiss
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from app.database.models import Book, User, Badge
from app.database.crud import create_book, get_books
from sqlalchemy import text
from dotenv import load_dotenv

load_dotenv()

MYSQL_USER = os.getenv('MYSQL_USER')
MYSQL_PASSWORD = os.getenv('MYSQL_PASSWORD')
MYSQL_HOST = os.getenv('MYSQL_HOST')
MYSQL_PORT = os.getenv('MYSQL_PORT')
MYSQL_DB = os.getenv('MYSQL_DB')

# 데이터베이스 연결 설정
DATABASE_URL = f"mysql+pymysql://{MYSQL_USER}:{MYSQL_PASSWORD}@{MYSQL_HOST}:{MYSQL_PORT}/{MYSQL_DB}"
engine = create_engine(DATABASE_URL)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

# 데이터베이스 세션 생성
mysql_db = SessionLocal()

In [3]:
result = mysql_db.execute(text("SHOW CREATE TABLE badges"))
table_structure = result.fetchone()[1]  # 결과에서 BADGE TABLE 구문 가져오기
print(table_structure)  # 테이블 생성 SQL 출력

CREATE TABLE `badges` (
  `badge_id` bigint NOT NULL AUTO_INCREMENT,
  `user_id` varchar(64) NOT NULL,
  `book_id` bigint NOT NULL,
  `badge_image` varchar(2048) NOT NULL,
  `created_at` timestamp NOT NULL DEFAULT (now()),
  PRIMARY KEY (`badge_id`),
  KEY `user_id` (`user_id`),
  KEY `book_id` (`book_id`),
  CONSTRAINT `badges_ibfk_1` FOREIGN KEY (`user_id`) REFERENCES `users` (`user_id`),
  CONSTRAINT `badges_ibfk_2` FOREIGN KEY (`book_id`) REFERENCES `books` (`book_id`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci


In [4]:
result = mysql_db.execute(text("SHOW CREATE TABLE users"))
table_structure = result.fetchone()[1]  # 결과에서 BADGE TABLE 구문 가져오기
print(table_structure)  # 테이블 생성 SQL 출력

CREATE TABLE `users` (
  `user_id` varchar(64) NOT NULL,
  `birth_year` varchar(4) NOT NULL,
  `gender` enum('M','F') NOT NULL,
  PRIMARY KEY (`user_id`),
  CONSTRAINT `chk_birth_year_format` CHECK (((length(`birth_year`) = 4) and regexp_like(`birth_year`,_utf8mb4'^[0-9]{4}$')))
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci


In [5]:
def alter_badge_table(db):
    try:
        # `badge_audio_url` 컬럼 삭제
        db.execute(text("ALTER TABLE badges DROP COLUMN badge_audio_url"))

        db.commit()
        print("🚀 [수정 완료] `badge_audio_url` 컬럼이 삭제되었습니다!")
    except Exception as e:
        db.rollback()
        print(f"❌ [오류 발생] {e}")

# 실행
alter_badge_table(mysql_db)

🚀 [수정 완료] `badge_audio_url` 컬럼이 삭제되었습니다!


In [10]:
def alter_users_table(db):
    try:
        # `name`, `phone_number`, `email` 컬럼 삭제
        db.execute(text("ALTER TABLE users DROP COLUMN name"))
        db.execute(text("ALTER TABLE users DROP COLUMN phone_number"))
        db.execute(text("ALTER TABLE users DROP COLUMN email"))

        db.commit()
        print("🚀 [수정 완료] `name`, `phone_number`, `email` 컬럼이 삭제되었습니다!")
    except Exception as e:
        db.rollback()
        print(f"❌ [오류 발생] {e}")

# 실행
alter_users_table(mysql_db)

❌ [오류 발생] (pymysql.err.OperationalError) (1091, "Can't DROP 'name'; check that column/key exists")
[SQL: ALTER TABLE users DROP COLUMN name]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
